In [14]:
from time import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pickle

In [2]:
df = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/train.csv')
df_test = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/test.csv')

df['length'] = df['user_review'].apply(lambda x: len(x))
df_test['length'] = df_test['user_review'].apply(lambda x: len(x))

In [3]:
df['title'].unique()

array(["Spooky's Jump Scare Mansion", 'Sakura Clicker', 'WARMODE',
       'Fractured Space', 'Dota 2', 'Path of Exile', 'War Thunder',
       'Ring of Elysium', 'Team Fortress 2', 'Yu-Gi-Oh! Duel Links',
       'SMITE®', 'Brawlhalla', 'World of Tanks Blitz',
       'DCS World Steam Edition', 'Heroes & Generals',
       'The Elder Scrolls®: Legends™', 'Trove', 'Neverwinter',
       'Realm Royale', 'PlanetSide 2', 'Realm of the Mad God', 'Elsword',
       'theHunter Classic', 'Eternal Card Game', 'Black Squad',
       'Freestyle 2: Street Basketball', 'Bless Online',
       'RaceRoom Racing Experience', 'Fallout Shelter',
       'Tactical Monsters Rumble Arena', 'Creativerse', 'Dreadnought',
       'Infestation: The New Z', 'Shop Heroes', 'Robocraft',
       'Business Tour - Board Game with Online Multiplayer',
       'Crusaders of the Lost Idols', 'AdventureQuest 3D',
       'Realm Grinder', 'World of Guns: Gun Disassembly',
       'Bloons TD Battles', 'Cuisine Royale', 'School of Drago

In [4]:
doc = ''
doc1 = ''
doc0 = ''
for i in range(df.shape[0]):
    doc = doc + df.loc[i,'user_review'] + str(' ')
    if(df.loc[i,'user_suggestion']==0):
        doc0 = doc0 + df.loc[i,'user_review']
    else:
        doc1 = doc1 + df.loc[i,'user_review']
n = 0

In [5]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

tokens = word_tokenize(doc)
all_words = [token.lower() for token in tokens if token.isalpha()!=0]
all_words = [word for word in all_words if word not in stop_words]

tokens0 = word_tokenize(doc0)
all_words0 = [token.lower() for token in tokens0 if token.isalpha()!=0]
all_words0 = [word for word in all_words0 if word not in stop_words]

tokens1 = word_tokenize(doc1)
all_words1 = [token.lower() for token in tokens1 if token.isalpha()!=0]
all_words1 = [word for word in all_words1 if word not in stop_words]

In [6]:
freq_all_words = FreqDist(all_words)
freq_all_words0 = FreqDist(all_words0)
freq_all_words1 = FreqDist(all_words1)

In [8]:
word_features = []
word_features0 = []
word_features1 = []

for i in range(1000):
    word_features0.append(freq_all_words0.most_common(1000)[i][0])
for i in range(1000):
    if(freq_all_words1.most_common(1000)[i][0] in word_features0):
        word_features0.remove(freq_all_words1.most_common(1000)[i][0])
        
for i in range(300):
    word_features.append(freq_all_words.most_common(300)[i][0])

word_features =  word_features + word_features0 + word_features1
word_features = list(set(word_features))
len(word_features)

493

In [10]:
for w in word_features:
    df[w] = df['user_review'].apply(lambda x: len(x.lower().split(w))-1)

symbol_features = ['!','.','\'',',','@','$','#',':',';','\"','&','*','%','^',' ']
for s in symbol_features:
    df[s] = df['user_review'].apply(lambda x: len(x.split(s))-1)

alpha_features = [chr(i) for i in range(ord('a'),ord('z')+1)]
for f in alpha_features:
    df[f] = df['user_review'].apply(lambda x: len(x.split(f))-1)
    
digit_features = [str(i) for i in range(10)]
for d in digit_features:
    df[d] = df['user_review'].apply(lambda x: len(x.split(d))-1)

In [11]:
for w in word_features:
    df_test[w] = df_test['user_review'].apply(lambda x: len(x.lower().split(w))-1)
    
for s in symbol_features:
    df_test[s] = df_test['user_review'].apply(lambda x: len(x.split(s))-1)

alpha_features = [chr(i) for i in range(ord('a'),ord('z')+1)]
for f in alpha_features:
    df_test[f] = df_test['user_review'].apply(lambda x: len(x.lower().split(f))-1)
    
digit_features = [str(i) for i in range(10)]
for d in digit_features:
    df_test[d] = df_test['user_review'].apply(lambda x: len(x.split(d))-1)

In [12]:
features = list(set(word_features + symbol_features + alpha_features + digit_features + ['length']))

In [ ]:
# for feature in features:
#     m  = df[feature].mean()
#     s = df[feature].std()
#     df[feature] = (df[feature] - m)/s
#     df_test[feature] = (df_test[feature] - m)/s

In [19]:
X = df[features]
# X = df[best_features]
y = df['user_suggestion']

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

for i  in range(4,5):
    print(i)
    if(i==0):
        model=LinearSVC()
    if(i==1):
        model=RandomForestClassifier()
    if(i==2):
        model=LogisticRegression(max_iter = 1000)
    if(i==3):
        model=XGBClassifier()
    if(i==4):
        model=GradientBoostingClassifier(n_estimators = 500)
    model.fit(X_train,y_train)

    train_score = f1_score(y_train, model.predict(X_train))
    test_score = f1_score(y_test, model.predict(X_test))
    print(train_score,test_score)

4
0.9047784173659755 0.8481958353202989


In [ ]:
# best_features = []
# for i in range(len(features)):
#     if(model.feature_importances_[i]>0.0005):
#         print(features[i],model.feature_importances_[i])
#         best_features.append(features[i])

In [20]:
filename = 'model'+'.sav'
pickle.dump(model, open(filename, 'wb'))

In [21]:
model.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [22]:
test_X = df_test[features]
# test_X = df_test[best_features]

df_test['user_suggestion'] = pd.Series(model.predict(test_X))
final = pd.concat([df_test['review_id'],df_test['user_suggestion']],axis=1)
final.to_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/nlp_sub1.csv',header=True , index=False)